# Bootstrap Calibrations

In [1]:
# imports
from desispec import bootcalib as desiboot
import numpy as np

## Find peaks

In [2]:
# Read flat
flat = fits.open('/Users/xavier/DESI/Wavelengths/pix-b0-00000001.fits')[0].data

In [25]:
reload(desiboot)
xpk, ypos = desiboot.find_fiber_peaks(flat)

Found 500 fibers
Found 20 bundles


## Trace the flat spectra

In [26]:
ypos

2048

### Crude

In [45]:
reload(desiboot)
xset, xerr = desiboot.trace_crude_init(flat,xpk[0:50],ypos)

### Polynomial fits (test)

In [78]:
from xastropy.xutils import afits as xafits

In [93]:
yval = np.arange(4096)
ii=4
xval = xset[:,ii]
gdval = xerr[:,ii] < 999.
dfit0 = xafits.func_fit(yval[gdval],xval[gdval],'legendre',6)

In [94]:
fitv = xafits.func_val(yval,dfit0)

In [95]:
xdb.xplot(yval,fitv, xtwo=yval,ytwo=xset[:,ii],mtwo='+')

In [96]:
xdb.xplot(yval[gdval],fitv[gdval]-xval[gdval], scatter=True)

### Polynomial fits (True)

In [111]:
reload(desiboot)
xfit, fits = desiboot.fit_traces(xset,xerr)#[:,0:5],xerr[:,0:5])

### QA

In [139]:
reload(desiboot)
desiboot.fiber_trace_qa(flat,xfit)

## PSF

### Sigma for each fiber (initial guess)